## 1.浏览器设置

In [37]:
# 禁用沙盒模式:add_argument('--no-sandbox')
# 保持浏览器打开状态:add_experimental option('detach'，True)# 创建并启动浏览器:webdriver.Chrome()
import time
import random
from selenium import webdriver #驱动浏览器
from selenium.webdriver.chrome.options import Options#用于设置谷歌浏览器
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# 创建设置浏览器对象
q1 = Options()
# 禁用沙盒模式
q1.add_argument('--no-sandbox')
#保持浏览器打开状态(默认是代码执行完毕自动关闭)
q1.add_experimental_option('detach', True) #保持浏览器打开状态
q1.add_argument("--disable-blink-features=AutomationControlled")  # 禁用自动化标志
q1.add_experimental_option('excludeSwitches', ['enable-automation'])  # 禁用自动化提示
q1.add_experimental_option("useAutomationExtension", False)
q1.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36")
#Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36

## 2.启动浏览器

In [38]:
if __name__ == "__main__":
    #创建并启动浏览器
    # Service调用驱动器，option设置浏览器
    driver = webdriver.Chrome(service=Service('chromedriver.exe'),options=q1)
    #隐藏 navigator.webdriver
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    #彻底隐藏自动化特征
    # 在页面加载前执行（通过 CDP）
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {
            "source": """
                Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
                window.navigator.chrome = undefined;
                Object.defineProperty(navigator, 'languages', {get: () => ['en-US', 'en']});
                Object.defineProperty(navigator, 'plugins', {get: () => [1, 2, 3]});
            """
        }
    )

    #WOS高级检索网址

    url = 'https://webofscience.clarivate.cn/wos/alldb/advanced-search'
 
    #请求网址
    driver.get(url)
    # 模拟人类操作
    time.sleep(random.uniform(2, 5))
    ActionChains(driver).move_by_offset(10, 10).perform()

## 3.自动化导出步骤设计

In [39]:
def export_all(start,end):
    
    # 点击【导出】//*[@id="export-trigger-btn"]/span[2]/span
    #driver.find_element(By.XPATH, '//*[@id="snRecListTop"]/app-export-menu/div/button/span[2]/span').click()
    driver.find_element(By.XPATH, '//*[@id="export-trigger-btn"]/span[2]/span').click()
    time.sleep(random.uniform(0.5, 1.5))
    # 点击【原文本文件】（//*[@id="exportToFieldTaggedButton"]）
    driver.find_element(By.XPATH, '//*[@id="exportToFieldTaggedButton"]').click()
    time.sleep(random.uniform(0.5, 1.5))
    # 勾选【Records from】（//*[@id="exportToFieldTaggedButton"]）
    driver.find_element(By.XPATH, '//*[@id="radio3"]/div/label').click()
    time.sleep(random.uniform(0.5, 1.5))
    #输入导出条数
    driver.find_element(By.XPATH,f'//*[@name="markFrom"]').clear()#清空开始数据
    time.sleep(random.uniform(0.5, 1.5))
    driver.find_element(By.XPATH,f'//*[@name="markFrom"]').send_keys(start)#输入开始数据
    time.sleep(random.uniform(0.5, 1.5))
    driver.find_element(By.XPATH,f'//*[@name="markTo"]').clear()#清空开始数据
    time.sleep(random.uniform(0.5, 1.5))
    driver.find_element(By.XPATH,f'//*[@name="markTo"]').send_keys(end)#输入开始数据
    time.sleep(random.uniform(0.5,1))
    #换成其他类型数据 //*[@id="primary-content"]/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button/span[1]
    driver.find_element(By.XPATH,f'//*[@id="primary-content"]/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button/span[1]').click()
    #driver.find_element(By.XPATH, '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button/mat-icon').click()
    time.sleep(random.uniform(0.5,1))
    
    #全记录 //*[@id="global-select"]/div/div/div[3]/span[1]
    driver.find_element(By.XPATH, '//*[@id="global-select"]/div/div/div[3]/span[1]').click() 
    time.sleep(random.uniform(0.5,1))

    #全记录与引用的参考文献
    #driver.find_element(By.XPATH, '//*[@id="global-select"]/div/div/div[4]/span').click()
    #time.sleep(random.uniform(1, 2))
    
    #点击导出
    driver.find_element(By.XPATH, '//*[@id="exportButton"]').click()
    # 显示等待/等待导出完成(“导出”按钮消失)
    try:
        WebDriverWait(driver, 150).until_not(
            EC.presence_of_element_located((By.XPATH, '//*[@id="exportButton"]'))
        )
        if start % 4000 == 0:
            time.sleep(random.uniform(30, 60))
    except:
        print("导出超时，请检查网络连接")

    time.sleep(random.uniform(1, 2))
#    driver.find_element(By.XPATH, '//*[@id="exportButton"]').click()
#    time.sleep(30)

## 4.1自动化导出（每次下载500条数据）

In [5]:
import os
from pathlib import Path

# Configuration
# 文件下载路径
downloads_path = Path(r"C:/Users/guyue/Downloads")
#文件保存路径
save_path = Path(r"C:\Users\guyue\Desktop\TOP_100")
old_file = downloads_path / "savedrecs.txt"
#导出出错时，修改s，更换起始点接着自动化下载数据
s = 10501

#爬取全记录与引用的参考文献（一次上限500条）
if __name__ == "__main__":
    #记录总数
    count_element = driver.find_element(By.CSS_SELECTOR, "div[data-ta-search-info-count]")
    # 获取data-ta-search-info-count属性的值
    all_counts = count_element.get_attribute("data-ta-search-info-count")
    print(f'共有{all_counts}条文献数据，现在开始导出……')

    for start in range(s,int(all_counts),500):
        end = start + 499
        if end > int(all_counts):
            end = int(all_counts)
        #异常处理
        try:
            export_all(start, end)
            time.sleep(1)
            if old_file.exists():
                print(f"已成功导出{start}-{end}条数据")
                new_file = save_path / f"savedrecs{start}_{end}.txt"
                old_file.rename(new_file)
                print(f"✓ 文件已重命名为: savedrecs{start}_{end}.txt")
            else:
                print(f"✗ 原文件不存在: {old_file},请检查下载文件夹和保存文件夹后修改s的值为{start}继续开始！！")
                break
        except Exception as e:
            print(f"✗导出{start}-{end}条数据时出错，请修改s的值为{start}继续开始！！")
            print(e)
            break

共有5480条文献数据，现在开始导出……


## 4.2自动化导出（每次下载1000条数据）

In [ ]:
import os
from pathlib import Path

# Configuration
# 文件下载路径
downloads_path = Path(r"C:\Users\guyue\Downloads")
# 文件保存路径,需要与下载路径在同一磁盘下，否则没有权限完成重命名文件
save_path = Path(r"C:\Users\guyue\Desktop\test")
old_file = downloads_path / "savedrecs.txt"
#导出出错时，修改s，更换起始点接着自动化下载数据
s = 1

#爬取全记录（一次上限1000条）
if __name__ == "__main__":
    #记录总数
    
    count_element = driver.find_element(By.CSS_SELECTOR, "div[data-ta-search-info-count]")
    # 获取data-ta-search-info-count属性的值
    all_counts = count_element.get_attribute("data-ta-search-info-count")
    print(f'共有{all_counts}条文献数据，现在开始导出……')

    #导出出错时，修改s，更换起始点接着自动化下载数据
    s = 1

    for start in range(s,int(all_counts),1000):
        end = start + 999
        if end > int(all_counts):
            end = int(all_counts)
        #异常处理
        try:
            export_all(start, end)
            time.sleep(1)
            if old_file.exists():
                print(f"已成功导出{start}-{end}条数据")
                new_file = save_path / f"savedrecs{start}_{end}.txt"
                old_file.rename(new_file)
                print(f"✓ 文件已重命名为: savedrecs{start}_{end}.txt")
            else:
                print(f"✗ 原文件不存在: {old_file},请检查下载文件夹和保存文件夹后修改s的值为{start}继续开始！！")
                break
        except Exception as e:
            print(f"✗导出{start}-{end}条数据时出错，请修改s的值为{start}继续开始！！")
            #print(e)
            break

共有5719条文献数据，现在开始导出……
已成功导出1-1000条数据
✓ 文件已重命名为: savedrecs1_1000.txt
已成功导出1001-2000条数据
✓ 文件已重命名为: savedrecs1001_2000.txt
已成功导出2001-3000条数据
✓ 文件已重命名为: savedrecs2001_3000.txt
✗导出3001-4000条数据时出错，请修改s的值为3001继续开始！！
